In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import precision_score,recall_score,accuracy_score

In [2]:
DATASET = "ENCODE"

columns_before_attributes = 4
frame = pd.read_csv("../Datasets/ENCODE/ENCODE_SPLIT.csv")

In [3]:
def create_classification_list(frame, columns_before_attributes):
    columns = frame.columns[columns_before_attributes:].values
    result_columns = []
    for col in columns:
        values_list = [col + "_" + x for x in list(set(frame[col].values))]
        result_columns.append(values_list)
    return result_columns

In [4]:
attributes_list = create_classification_list(frame, columns_before_attributes)
limits = []
tot_length = 0
for attr in attributes_list:
    tot_length += len(attr)
    limits.append(tot_length)
    

In [5]:
file = pd.read_csv("Data/manually_labelled_gsm_data.csv")

In [6]:
limits

[26,
 35,
 379,
 390,
 819,
 822,
 827,
 844,
 1013,
 1019,
 1029,
 1044,
 1109,
 1115,
 1137]

In [7]:
predictions = pd.read_csv("Results/manually_labelled_gsm_results_RoBERTa_69epochs.csv")

In [8]:
attribute_names = ['Assay name','Assay type','Target of assay','Genome assembly','Biosample term name','Project','Organism','Life stage','Age','age units','Sex','Ethnicity','Health status','Classification','Investigated as']
attributes = ['assay name: ','assay type: ','target of assay: ','genome assembly: ','biosample term name: ','project: ','organism: ','life stage: ','age: ','age units: ','sex: ','ethnicity: ','health status: ','classification: ','investigated as: ']

In [9]:
print("Iterating over test_set lines...")
robe_predicted_values = []
for i, item in predictions.iterrows():
    float_prediction = np.array(predictions.iloc[i].values)
    predicted_indexes = []
    #Take the last 4 elements of the argosrt (indices that where = 1) and sort the result
    for i in range(len(attributes)):
        if(i== 0):
            predictions_for_attribute = float_prediction[:limits[i]]
            max_index = np.argmax(predictions_for_attribute)
        elif(i == len(attributes) - 1):
            predictions_for_attribute = float_prediction[limits[i - 1]:]
            max_index = np.argmax(predictions_for_attribute) + limits[i - 1]
        else:
            predictions_for_attribute = float_prediction[limits[i-1]:limits[i]]
            max_index = np.argmax(predictions_for_attribute) + limits[i - 1]
        predicted_indexes.append(max_index)
    robe_predicted_values.append(predicted_indexes)

Iterating over test_set lines...


In [13]:
roberta_text_pred = []
for pred in robe_predicted_values:
    tmp = ""
    for i in range(len(pred)):
        if(i>0):
            tmp = tmp + attributes_list[i][pred[i] - limits[i-1] -1].replace("_", ": ") + ' - '
        else:
            tmp = tmp + attributes_list[i][pred[i]].replace("_", ": ") + ' - '
    roberta_text_pred.append(tmp)

In [14]:
results = pd.DataFrame([[x,y] for (x,y) in zip(file.GSM, roberta_text_pred)])

In [20]:
results.to_csv("Results/manually_labelled_gsm_text_results_Roberta_69epochs.csv", index = False)